In [1]:
import os
import pandas as pd

## Face vs. no-Face Scene Events

In [2]:
run = 8

In [3]:
deriv_dir = "/home/exp-psy/Desktop/study_face_tracks/derivatives"
df_path = os.path.join(deriv_dir, "reference_face-svm", f"run-0{run}_no-faces.tsv")

In [4]:
in_df = pd.read_csv(df_path, sep="\t", index_col=0)
in_df.head(10)

,onset,duration,trial_type
0,0.0,1.0,no_face_1
1,2.0,7.0,no_face_2
2,75.0,12.0,no_face_3
3,88.0,1.0,no_face_4
4,102.0,4.0,no_face_5
5,108.0,3.0,no_face_6
6,134.0,6.0,no_face_7
7,175.0,30.0,no_face_8
8,223.0,1.0,no_face_9
9,249.0,2.0,no_face_10


## Start with no-Face Scene Events

In [5]:
for run in range(1, 9):
    df_adj = in_df[in_df["duration"] > 1].copy()
    # df_adj["duration"] = 2.0
    df_adj["trial_type"] = df_adj["trial_type"].str.replace("_", "", regex=False)
    df_adj.to_csv(
        os.path.join(deriv_dir, "reference_face-svm", f"run-0{run}_adjusted-no-faces.tsv"), 
        sep="\t", 
        index=False
    )

In [6]:
df_adj.head(10)

,onset,duration,trial_type
1,2.0,7.0,noface2
2,75.0,12.0,noface3
4,102.0,4.0,noface5
5,108.0,3.0,noface6
6,134.0,6.0,noface7
7,175.0,30.0,noface8
9,249.0,2.0,noface10
10,253.0,7.0,noface11
11,266.0,17.0,noface12
13,290.0,9.0,noface14


## Continue with overall Face Events

In [7]:
for run in range(1, 9):
    in_df["end"] = in_df["onset"] + in_df["duration"]
    # sort by onset just in case
    in_df = in_df.sort_values(by="onset").reset_index(drop=True)

    face_segments = []
    prev_end = in_df.iloc[0]["end"]

    for i in range(1, len(in_df)):
        onset = in_df.iloc[i]["onset"]

        if onset > prev_end:
            face_segments.append({
                "onset": prev_end,
                "duration": onset - prev_end,
                "trial_type": f"face{len(face_segments)+1}"
            })

        prev_end = max(prev_end, in_df.iloc[i]["end"])

    df_faces = pd.DataFrame(face_segments)
    df_faces_adj = df_faces[df_faces["duration"] > 1].copy()
    # df_faces_adj["duration"] = 2
    df_faces_adj.to_csv(
        os.path.join(deriv_dir, "reference_face-svm", f"run-0{run}_adjusted-faces.tsv"), 
        sep="\t", 
        index=False
    )

In [8]:
df_faces_adj.head(10)

,onset,duration,trial_type
1,9.0,66.0,face2
3,89.0,13.0,face4
4,106.0,2.0,face5
5,111.0,23.0,face6
6,140.0,35.0,face7
7,205.0,18.0,face8
8,224.0,25.0,face9
9,251.0,2.0,face10
10,260.0,6.0,face11
11,283.0,4.0,face12


## Face Identity Events

In [9]:
df_path_frame = os.path.join(deriv_dir, "reference_face-svm", f"run-0{run}_per-frame-faces.tsv")

In [10]:
in_df_frame = pd.read_csv(df_path_frame, sep="\t", index_col=0)
in_df_frame.head(10)

,duration,onset,object_id,FaceRecognitionFaceLocationsExtractor#face_locations
0,1.0,1.0,0,"(194, 611, 364, 441)"
1,1.0,9.0,0,"(263, 1058, 433, 888)"
2,1.0,10.0,0,"(246, 1041, 416, 871)"
3,1.0,11.0,0,"(246, 1024, 416, 854)"
4,1.0,12.0,0,"(246, 1006, 416, 836)"
5,1.0,13.0,0,"(246, 1006, 416, 836)"
6,1.0,14.0,0,"(246, 1006, 416, 836)"
7,1.0,15.0,0,"(246, 1041, 416, 871)"
8,1.0,15.0,1,"(129, 215, 373, 0)"
9,1.0,16.0,0,"(263, 1075, 433, 905)"


In [11]:
in_df_frame.object_id.unique()

array([0, 1, 2, 3])

In [12]:
for run in range(1, 9):
    segments = []
    start_idx = 0

    for i in range(1, len(in_df_frame)):
        current = in_df_frame.loc[i]
        previous = in_df_frame.loc[i - 1]

        # check if consecutive onset and same object_id
        if current["onset"] != previous["onset"] + 1 or current["object_id"] != previous["object_id"]:
            # only keep segments with more than 1 frame
            if i - start_idx > 1:
                segment = in_df_frame.iloc[start_idx:i]
                object_id = segment["object_id"].mode()[0]
                segments.append({
                    "onset": segment["onset"].iloc[0],
                    "duration": segment["onset"].iloc[-1] - segment["onset"].iloc[0] + 1,
                    "trial_type": f"face_{int(object_id)}"
                })
            start_idx = i

    # handle the last segment
    if len(in_df_frame) - start_idx > 1:
        segment = in_df_frame.iloc[start_idx:]
        object_id = segment["object_id"].mode()[0]
        segments.append({
            "onset": segment["onset"].iloc[0],
            "duration": segment["onset"].iloc[-1] - segment["onset"].iloc[0] + 1,
            "trial_type": f"face_{int(object_id)}"
        })
    df_face_identy = pd.DataFrame(segments)
    
    # finally save the df
    df_faces_adj.to_csv(
        os.path.join(deriv_dir, "reference_face-svm", f"run-0{run}_face-identity.tsv"), 
        sep="\t", 
        index=False
    )

In [13]:
df_face_identy.head(10)

,onset,duration,trial_type
0,9.0,7.0,face_0
1,21.0,44.0,face_0
2,66.0,4.0,face_0
3,73.0,2.0,face_0
4,89.0,4.0,face_0
5,98.0,2.0,face_0
6,100.0,2.0,face_0
7,129.0,2.0,face_0
8,131.0,2.0,face_0
9,140.0,2.0,face_0
